In [2]:
from collections import Counter
import pandas as pd
import numpy as np
from fuzzywuzzy import process
import re

In [14]:
df_bank = pd.read_csv('bank_statement.csv')
df_ckt = pd.read_csv('checkout.csv')

In [15]:
common_bank = [word[0] for word in Counter(" ".join(df_bank["desc"]).split()).most_common(15)]
common_ckt = set([word[0] for word in Counter(" ".join(df_ckt["buyer_name"]).split()).most_common(300)])

common_bank = [word for word in common_bank if word != '.']

for i, word in enumerate(common_bank):
    if '.' == word:
        common_bank[i] = ''

    common_bank[i] = re.sub('\.', '\.', word)
    common_bank[i] = re.sub('\+', '\+', word)
    common_bank[i] = re.sub('\*', '\*', word)

In [16]:
df_bank['desc'] = df_bank['desc'].str.replace('\.', '').str.replace("|".join(common_bank), '', regex=True).str.strip().str.split().apply(lambda x: ' '.join(x))

In [17]:
df_bank['desc']

0                     BW CGNBWI YIWHI LEZC HX ZLLEZC HX
1                                    WOOG CRIZ NIWSI YO
2                             ZOMZW MBIX TM BUGW KZBUGW
3                     GO L/F HH PUYIM YEBF GN BYYEBF GN
4                     FBZUN WIQT JIW F TEGF PY TTEGF PY
                              ...                      
239995                          RV WC XUH QLIX GIXYI WX
239996           RO M/B EYEKI T GXZMIB YELB RX YYELB RX
239997               PUZ JTITI WJJIZJ TEDW HZ DTTEDW HZ
239998    WUVIJ XJVIS FISD SURWISZ POYER KSURWISZ POYER
239999             WEE BIJ SUI BCC QD LV XLDZ XBLV XLDZ
Name: desc, Length: 240000, dtype: object

In [18]:
max_statement = df_bank["stmt_amount"].max()
min_statement = df_bank["stmt_amount"].min()
group_bank = df_bank.sort_values(["stmt_amount"],ascending=True).groupby(["stmt_amount"])
group_ckt = df_ckt.sort_values(["ckt_amount"],ascending=True).groupby(["ckt_amount"])

In [31]:
def fuzzy_merge(df_1, df_2, key1, key2, thresh=40, limit=1):
    query = df_2[key2].tolist()
    index = df_2['ckt_id'].tolist()

    mapper = dict()

    for q, i in zip(query,index):
        mapper[q] = i
    
    match = df_1[key1].apply(lambda x: process.extract(x, query, limit=limit))
    df_1['matches'] = match
    
    filtered_match = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x]))
    ckt_id = filtered_match.apply(lambda x: mapper[x])

    df_1['matches'] = filtered_match
    df_1['ckt_id'] = ckt_id
    
    
    return df_1

In [32]:
total = 0
final_df = pd.DataFrame()
for bank, ckt in zip(group_bank, group_ckt):
    # kamus name checkout, fuzzywuzzy ama bank
    res = fuzzy_merge(bank[1], ckt[1], 'desc', 'buyer_name')

    res.drop(['stmt_amount', 'desc', 'matches'], inplace=True)

    # klo dpt match 1 ato 2 lgsg join aja

print(total)

stmt_id  stmt_amount                                          desc  \
37740     37741         0.01                              BUDBBW ZUGH ZIFY   
150536   150537         0.01                  PHK I PIHK GEPC TY PGGEPC TY   
226724   226725         0.01                                   WOK PEE PQB   
230963   230964         0.01  NOL WQIN FWXNH VUQINR GOFONR VTVUQINR GOFONR   
115857   115858         0.01                 IBXILY CIBKI CP BULVIBJ CL NG   
57353     57354         0.01                NMH SJTUY HIMHW SIZ I NMH WNMH   
39780     39781         0.01              KOOVWYIXIK MIKSI XI GIK PRXI GIK   
25601     25602         0.01                  POBIGEPGQTI PUWPPM SINYI SLY   
127225   127226         0.01                   QOVCI PICRI PCY SUCF MWSUCF   
169611   169612         0.01             FHUK VHIX FIT ZIQ GYZETD GZGYZETD   
194236   194237         0.01           XUT QDBIB WIXDI N GEPY DT PGGEPY DT   
201904   201905         0.01                          GQ DUX RMXDI ZIDGI J   
36484     36485         0.01                        NEBW BUV KIQZ LIBKI IH   
131899   131900         0.01                 BJIB MOLT KSIL PS IYVH YBIYVH   
144531   144532         0.01                        UQUR WUVUS PMIXX ZIWMI   
66243     66244         0.01                     YPXKQEP BORBG ZBPEE JIX B   
163354   163355         0.01              NOQNT DTQIW MIL NEVS ZR NNEVS ZR   
83507     83508         0.01                            GTIBB QIGYI VOBC V   
140077   140078         0.01                            DWHIS GIMPI NSDN D   
93061     93062         0.01                            LOQFNNEN MILZI YOG   
156287   156288         0.01      JFF ZLLEWJB XM JOWJB WRIYRI LEWJB JLEWJB   
75534     75535         0.01                       JGEP SGEMH GI JSG WYJSG   
145978   145979         0.01           GIOWI IRI XEJEHHEX ZEJH SSSQ ZEHISO   
181381   181382         0.01        SEVK TIDI TUQD GID G SEVK ZY VSSEVK ZY   
114359   114360         0.01              WXKIH CIN RXVN REGZ BT GRREGZ BT   

                                  matches  ckt_id  
37740    BUDABBAW ZAUGAH ZIFYI XIH AXIWIH  183132  
150536                       PAHK AI PIHK  229799  
226724                       WOK PEE PAQB  305879  
230963  NOL WAQIN FAWXANAH ZINQI AXAKUSIN  127069  
115857                IBAXILAY ACP LAYGAB   48817  
57353             SAJTUY HIMHAW SIZ IMHAL   17789  
39780         KOOVAWYIXIK MIKSI AMHUDDAF    14771  
25601                PUWAPPAM SINYI ASLAY  300720  
127225                  QOVCI PICRI APACY    6222  
169611             AFHUK VAHIX FIT ZIQNAT  154751  
194236           XUT QADBIBA WIXDI NAAQAT  281850  
201904        DUX RAMAXADI ZIDGI RUWVICWI   95499  
36484              BUV AKIQAZ LIBKI IHABI  285819  
131899                        BAJIAB MOLT  216111  
144531                      WUVUS PAMIXAX  227279  
66243            BORABAG ZABPEE JIX SAPIB  289456  
163354           NOQANAT DATQIW MIL QUSIL  250355  
83507            GATIBAB QIGYI VOBC WACLI    2386  
140077            DAWHIAS GIMPI NSADNUHIM  234781  
93061           LOQFANENNA MILZI YOGT LOQ  298747  
156287             XAM JOWAJAB WARIYRILDI  298686  
75534                      JGEP SGEMH GAI   69921  
145978                    SASASQ ZEHITOSO  273052  
181381              TAIDI ATUQAD G GAKEYI   53390  
114359                  WXAKIH CIN RAXVAN  106443

0
